# Modeling Sequential Data Using Recurrent Neural Networks

In this chapter, we will explore **RNNs** and see their application in modeling sequential data and a specific subset of sequential data---time-series data.  In this chapter, we will cover the following topcis:
- Introducing Sequential Data
- RNNs for modeling sequences
- Long Short-Term Memory (LSTM)
- Truncated Backpropagation through time (T-BPTT)
- Implementing a multilayer RNN for sequence modeling in TensorFlow
- Project one: RNN sentiment analysis of the IMDb movie review dataset
- Project two: RNN character-level language modeling with LSTM cells, using text data from Shakespeare's Hamlet
- Using gradient clipping to avoid exploding gradients

## Modeling sequential data---Order Matters

What makes sequences unique from other data types is that elements in a sequence appear in a certain order, and are not independent of each other.  Order matters.

## Representing Sequences

Throughout this chapter, we will represent sequences as $\left( \mathbf{x}^{(1)}, \mathbf{x}^{(2)}, \ldots, \mathbf{x}^{(T)} \right)$.  The superscript indices indicate the order of the instances, and the length of the sequence is $T$.

The following figure shows and example of time-series data where both $x$'s and $y$'s naturally follow the order according to their time axis; therefore, both $x$'s and $y$'s are sequences:

<img src="images/16_01.png" style="width:500px">

RNNs are designed for modeling sequences and are capable of remembering past information and processing new events accordingly.

## The different categories of sequence modeling

Sequence modeling has many fascinating applications, such as language translation, image captioning, and text generation.

We need to uderstand the different types of sequence modeling tasks to develop an appropriate model.  The following figure shows several different relationship categories of input and output data:
<img src="images/16_02.png" style="width:500px">

If either the input or output data is a sequence, the data will form one of the following three different categories:
- **Many-to-One**: The input data is a sequence, but the output is a fixed-size vector, not a sequence.  For example, in sentiment analysis, the input is text-bases and the output is a class label.
- **One-to-Many**: The input data is in standard format, not a sequence, but the output is a sequence.  An example of this category is image captioning---the input is an image; the output is an English phrase.
- **Many-to-Many**: Both the input and the output arrays are sequences.  This category can be further divided based on whether the input and output are synchronized or not.  An example of a **synchronized** many-to-many modeling task is video classification, where each frame in a vdeo is labeled.  An example of a **delayed** many-to-many modeling taks is translating a language into another language.  For instance, an entire English sentence must be read and processed by a machine before producing its translation into German.

## RNNs for modeling sequences

In this section, now that we understand sequences, we can look at the foundations of RNNs.  We'll start by introducing the typical structure of an RNN, and we'll see how the data flows through it with one or more hidden layers.  We'll then examine how the neuron activations are computed in a typical RNN.  This will create a context for us to discuss the common challenges in training RNNs, and explore the modern solution to these challenges---LSTM.

## Understanding the structure and flow of an RNN

<img src="images/16_03.png" style="width:500px">
In a standard feedforward network, information flows from the input to the hidden layer, and then from the hidden layer to the output layer.  On the other hand, in a recurrent network, the hidden layer gets its input from both the input layer and the hidden layer from the previous time step.

The folow of information in adjacent time steps in the hidden layer allows the network to have a memory of past events.  This flow of information is usually displayed as a loop, also known as a recurrent edge in graph notation, which is how this general architecture got its name.

In the following figure, the single hedden layer network and the multilayer network illustrate two contrasting architectures:

<img src="images/16_04.png" style="width:500px">
In order to examine the architecture of RNNs and the flow of information, a compact representation with a recurrent edge can be unfolded, which you can see in the preceding figure.

As we know, each hidden unit in a standard neural network receives only one input---the net preactivation associated with the input layer.  Now, in contrast, each hidden unit in an RNN receives two distinct sets of input---the preactivation from the input layer and the activation of the same hidden layer from the previous time step $(t-1)$.

## Computing activations in an RNN

Now that we understand the structure and general flow of information in an RNN, let's get more specific and compute the actual activations of the hidden layers as well as the output layer.  For simplicity, we'll consider just a single hidden layer; however, the same concept applies to multilayer RNNs.

Each directed edge (the connections between boxes) in the representation of an RNN that we just looked at is associated with a weight matrix.  Those weights do not depend on time $t$; therefore, they are shared across the time axis.  The different weight matrices in a single layer RNN are as follows:
- $W_{xh}$: The weight matrix between the input $\mathbf{x}^{t}$ and the hidden layer $\mathbf{h}$.
- $W_{hh}$: The weight matrix associated with the recurrent edge.
- $W_{hy}$: The weight matrix between the hidden layer and the output layer.

You can see these weight matrices in the following figure:

<img src="images/16_05.png" style="width:500px">

In certain implementations, you may observe that the weight matrices $\mathbf{W}_{xh}$ and $\mathbf{W}_{hh}$ are concatenated to a combined matrix $\mathbf{W}_h = \left[ \mathbf{W}_{xh}; \mathbf{W}_{hh} \right]$.  Later on, we will make use of this notation as well.

Computing the activations is very similar to standard multilayer perceptrons, and other types of feedforward nerual networks.  For the hidden layer, the net input $\mathbf{z}_{h}$ (preactivation) is:
$$
\mathbf{z}_h^{(t)} = \mathbf{W}_{xh}\mathbf{x}^{(t)} + \mathbf{W}_{hh}\mathbf{h}^{(t-1)} + \mathbf{b}_h
$$

Then the activation of the hidden units at the time step $t$ are calculated as follows:
$$
\mathbf{h}^{(t)} = \phi_h\left( \mathbf{W}_{xh}\mathbf{x}^{(t)} + \mathbf{W}_{hh}\mathbf{h}^{(t-1)} + \mathbf{b}_h \right)
$$
where $\mathbf{b}_h$ is the bias vector for the hidden uniits, and $\phi_h(\cdot)$ is the activation function of the hidden layer.

The activations of the output units will be computed as follows:
$$
\mathbf{y}^{(t)} = \phi_y\left( \mathbf{W}_{hy}\mathbf{h}^{(t)} + \mathbf{b}_y \right)
$$

To help clarify this further, the following figure shows the process of computing these activtions with both formulations:

<img src="images/16_06.png" style="width:500px">

## The challenges of learning long-range interactions

Backpropagation through time, or BPTT, which we briefly mentioned in the previous information box, introduces some new challenges.

In computing the gradients of a loss function, the so-called **vanishing** or **exploding** gradient problem arises.  This problem is explained through the examples in the following figure, which shows an RNN with only one hidden unit for simplicity:
<img src="images/16_07.png" style="width:500px">

In practice, there are two solutions to this problem:
- Truncated Backpropagation through time (TBPTT)
- Long short-term memory (LSTM)

TBPTT clips the gradients above a given threshold.  While TBPTT can solve the exploding gradient problem, the truncation limits the number of steps that the gradient can effectively flow back and properly update the weights.

On the other hand, LSTM has been more succussful in modeling long-range sequences by overcoming the vanishing gradient problem.  Let us discuss the LSTM in more detail:

## LSTM units

The building block of an LSTM is a memory cell which essentially represents the hidden layer.

In each memory cell, there is a recurrent edge that has the desirable weight $w=1$.  The values associated with this recurrent edge is called the **cell state**.  The unfolded structure of a modern LSTM is shown in the following figure:

<img src="images/16_18.png" style="width:500px">

Notice that the cell state from the previous time step, $\mathbf{C}^{(t-1)}$, is modified to get the cell state at the current time step, $\mathbf{C}^{(t)}$, without being multiplied directly with any weight factor.

Notice that $\odot$ refers to the element-wise product, while $\oplus$ means element-wise summation.

In the figure, four boxes are indicated with an activation function, either the sigmoid function ($\sigma$), or the hyperbolic tangent (tanh), and a set of weights.  These units of computation with sigmoid activation functions, whose output are passed through $\odot$ are called **gates**.

In an LSTM cell, there are 3 different types of gates, known as the forget gate, the input gate, and the output gate:
- The **forget gate** ($\mathbf{f}_t$), allows the memory cell to reset the cell sate, without growing indefintely.  In fact, the forget gate decides which information is allowed to go through and which information to suppress.
$$
\mathbf{f}_t = \sigma\left( \mathbf{W}_{xf}\mathbf{x}^{(t)} + \mathbf{W}_{hf}\mathbf{h}^{(t-1)} + \mathbf{b}_f \right)
$$
- The input gate ($\mathbf{i}_t$) and the input node ($\mathbf{g}_t$) are responsible for updating the cell state.  The are computed as follows:
$$
\mathbf{i}_t = \sigma\left( \mathbf{W}_{xi}\mathbf{x}^{(t)} + \mathbf{W}_{hi}\mathbf{h}^{(t-1)} + \mathbf{b}_i \right)
$$
$$
\mathbf{g}_t = \tanh\left( \mathbf{W}_{xg}\mathbf{x}^{(t)} + \mathbf{W}_{hg}\mathbf{h}^{(t-1)} + \mathbf{b}_g \right)
$$

The cell state at time $t$ is then computed as follows:
$$
\mathbf{C}^{(t)} = \left( \mathbf{C}^{(t-1)}\odot\mathbf{f}_t \right)\oplus\left( \mathbf{i}_t\odot\mathbf{g}_t \right)
$$
- The output gate $\left( \mathbf{o}_t \right)$ decides how to update the values of the hidden units:
$$
\mathbf{o}_t = \tanh\left( \mathbf{W}_{xo}\mathbf{x}^{(t)} + \mathbf{W}_{ho}\mathbf{h}^{(t-1)} + \mathbf{b}_o \right)
$$
Given this, the hidden units at the current time step are computed as follows:
$$
\mathbf{h}^{(t)} = \mathbf{o}_{t}\odot\tanh\left( \mathbf{C}^{(t)} \right)
$$

## Implmenting a multilayer RNN for sequence modeling in TensorFlow

Now that we introduced the underlying theory behind RNNs, we are ready to move on to the more practical part to implement RNNs in TensorFlow.  During this rest of this chapter, we will apply RNNs to 2 common problem tasks:
1. Sentiment analysis
2. Language modeling

## Project one---performing sentiment analysis of IMDb movie reviews using multilayer RNNs

In this section and the following subsections, we will implement a multilayer RNN for sentiment analysis using a many-to-one architecture.

In the next section, we will implement a many-to-many RNN for an application of language modeling.

## Preparing the data

In the preprocessing steps in _Chapter 8_, we created a clean dataset named `movie_data.csv`, which we'll use again now.  So let us import the necessary modules and read the data into a pandas `DataFrame`, as follows:

In [1]:
import pyprind
import pandas as pd
from string import punctuation
import re
import numpy as np

In [2]:
df = pd.read_csv("movie_data.csv", encoding="utf-8")

Recall that this `df` data frame has 2 columns, name `'review'` and `'sentiment'`, where `'review'` contains the text of movie reviews and `'sentiment'` contains the `0` or `1` labels.  The text component of these movie reviews are sequences of words; therefore, we want to build an RNN model to process the words in each sequence, and at the end, classify the entire sequence to `0` or `1` classes.

To prepare the data for input to a neural network, we need to encode it into numeric values.  To do this, we first find the unique words in the dataset, which can be done using sets in Python.  However, using sets for such a large dataset is _not_ efficient.  A more efficient way is to use `Counter` from the collections package.  The documentation for counter can be found at https://docs.python.org/3/library/collections.html#collections.Counter

In the following code, we will define `counts` object from the `Counter` class that collects the counts of occurrence of each unique word in the text.  Note that in this particular application (and in contrast to the bag-of-words model), we are only interested in the set of unique words, and won't require the word counts, which are created as a side product.

Then, we create a mapping in the form of a dictionary that maps each unique word, in our dataset, to a unique integer number.  We call this dictionary `word_to_int`, which can be used to convert the entire text of a review into a list of numbers.  The unique words are sorted based on their counts, but any any arbitary order can be used without affecting the final results.  This process of converting a text into a list of numbers is performed using the following code:

In [3]:
## Preprocessing the data: Separate the words and count each word's occurrence

from collections import Counter

counts = Counter()
pbar = pyprind.ProgBar(len(df["review"]), title="Counting words occurrences")
for i, review in enumerate(df["review"]):
    text = "".join([c if c not in punctuation else " "+c+" " for c in review]).lower()
    df.loc[i, "review"] = text
    pbar.update()
    counts.update(text.split())

Counting words occurrences
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:05:37


In [4]:
print(type(counts))
print(counts.get("the"))

<class 'collections.Counter'>
667950


In [9]:
## Create a mapping:  Map each unique word to an integer
word_counts = sorted(counts, key=counts.get, reverse=True)  # descending order
print(word_counts[:10])
word_to_int = {word: j for j, word in enumerate(word_counts, 1)}
print(word_to_int)

['the', '.', ',', 'and', 'a', 'of', 'to', "'", '/', 'is']
{'the': 1, '.': 2, ',': 3, 'and': 4, 'a': 5, 'of': 6, 'to': 7, "'": 8, '/': 9, 'is': 10, '>': 11, '<': 12, 'br': 13, 'it': 14, 'in': 15, 'i': 16, 'this': 17, 'that': 18, '-': 19, '"': 20, 's': 21, 'was': 22, 'as': 23, 'movie': 24, 'for': 25, 'with': 26, 'but': 27, 'film': 28, ')': 29, '(': 30, 'you': 31, 't': 32, 'on': 33, 'not': 34, 'he': 35, 'are': 36, 'his': 37, 'have': 38, 'one': 39, 'be': 40, '!': 41, 'all': 42, 'at': 43, 'they': 44, 'by': 45, 'an': 46, 'who': 47, 'so': 48, 'from': 49, 'like': 50, 'there': 51, 'or': 52, 'just': 53, 'her': 54, 'out': 55, 'about': 56, 'if': 57, 'has': 58, '?': 59, 'what': 60, 'some': 61, 'good': 62, 'can': 63, 'when': 64, 'more': 65, 'very': 66, 'she': 67, 'up': 68, 'no': 69, 'time': 70, 'my': 71, 'even': 72, 'would': 73, 'which': 74, 'only': 75, 'story': 76, 'really': 77, 'see': 78, 'their': 79, 'had': 80, 'me': 81, 'well': 82, 'we': 83, 'were': 84, ':': 85, 'than': 86, 'much': 87, 'bad': 88

In [6]:
a = [word for word in df.loc[1, "review"].split()]
# print(a)
# print(df.loc[1, "review"])

In [7]:
mapped_reviews = []
pbar = pyprind.ProgBar(len(df['review']), title="Map reviews to ints")
for review in df["review"]:
    mapped_reviews.append([word_to_int[word] for word in review.split()])
    pbar.update()

Map reviews to ints
0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:00:02


In [8]:
print(mapped_reviews[:2])

[[15, 5646, 3, 1, 2160, 3977, 26959, 30, 4824, 1575, 29, 1133, 7, 1, 316, 19, 720, 1612, 6, 6653, 753, 3, 14781, 3, 8680, 2, 33, 1, 14782, 328, 3, 3612, 6, 2105, 3, 67, 22, 1947, 15, 1, 8755, 6, 54, 339, 4, 54, 602, 4712, 15343, 2, 1732, 19, 121, 170, 323, 3, 1, 565, 953, 25748, 30, 1464, 19622, 29, 3, 47, 10, 5, 1121, 1099, 1362, 18, 58, 3040, 15, 6002, 25, 44343, 15, 716, 2, 1544, 2, 5290, 3192, 4, 1670, 7, 21518, 3, 1103, 7, 3921, 1, 434, 26, 37, 1956, 1801, 2333, 30, 3679, 3294, 29, 26, 1, 1285, 6, 507, 5, 286, 2, 1, 5463, 10558, 4, 92, 34, 2568, 106, 3, 27, 26, 1, 1457, 6, 1, 5056, 1362, 1320, 8833, 30, 623, 12704, 29, 18, 22, 15, 2762, 6, 1, 3488, 15, 1, 1358, 8, 21, 3, 44, 1873, 1, 1795, 4, 5, 5000, 6, 661, 4, 308, 7, 999, 1, 602, 2, 12, 13, 9, 11, 12, 13, 9, 11, 20, 602, 15, 14781, 20, 10, 5, 62, 255, 24, 3, 26, 1, 307, 76, 6, 5, 602, 6, 5, 3521, 170, 174, 260, 18, 22, 2373, 45, 5, 3041, 2160, 654, 412, 22, 5, 3129, 2, 1, 976, 4, 994, 244, 347, 79, 2413, 7, 999, 1, 602, 25, 65,

There is however one issue that we still need to solve---the sequences currently have different lengths.  In order to generate input data that is compatible with our RNN architecture, we will need to make sure that all the sequences have the same length.

For this purpose, we define a parameter called `sequence_length` that we set to 200.  Sequences that have than 200 words will be left-padded with zeros.  Vice versa, sequences that are longer than 200 words are cut such that only the last 200 corresponding words will be used.  We can implement this preprocessing step in two steps:
1. Create a matrix of zeros, where each row corresponds to a sequence of size 200.
2. Fill the index of words in each sequence from the right-hand side of the matrix.  Thus, if a sequence has a length of 150, the first 50 elements of the corresponding row will stay zero.

These 2 steps are shown in the following figure, for a small example with 8 sequences of sizes 4, 12, 8, 11, 7, 3, 10, 13.

<img src="images/16_09.png" style="width:500px">

Note that `sequence_length` is, in fact, a hyperparameter, and can be tuned for optimal performance.  We encourage you to fine-tune this parameter with different values, such as 50, 100, 200, 250, and 300.

Check out the following code for the implementation of these steps to create sequences of the same length:

In [10]:
# Define same-length sequences.
# If the sequence length < 200:  left-pad with zeros.
# If the sequence length > 200:  use the LAST 200 elements.

sequence_length = 200  # known as T in our RNN formulas.
sequences = np.zeros((len(mapped_reviews), sequence_length), dtype=int)

for i, row in enumerate(mapped_reviews):
    review_arr = np.array(row)
    sequences[i, -1*len(row):] = review_arr[-1*sequence_length:]

After we preprocess the dataset, we can proceed with splitting the data into separate training and test sets.  Since the dataset was already shuffled, we can simply take the first half of the dataset for training, and the second half for testing, as follows:

In [11]:
X_train = sequences[:25000, :]
y_train = df.loc[:25000, "sentiment"].values
X_test = sequences[25000:, :]
y_test = df.loc[25000:, "sentiment"].values

Now, if we want to separate the dataset for cross-validation, we can further split the second half of the data further to generate a smaller test set and a validation set for hyperparameter optimization.

Finally, we define a helper function that breaks a given dataset (which could be a training set or test set) into chunks, and returns a generator to iterate through these chunks (also known as mini-batches):

In [31]:
np.random.seed(123)  # for reproducibility

# Define a function to generate mini-batches:
def create_batch_generator(x, y=None, batch_size=64):
    n_batches = len(x)//batch_size
    x = x[:n_batches*batch_size]
    if y is not None:
        y = y[:n_batches*batch_size]
    for ii in range(0, len(x), batch_size):
        if y is not None:
            yield x[ii:ii+batch_size], y[ii:ii+batch_size]
        else:
            yield x[ii:ii+batch_size]

Using generators, as we've done in this code, is a very useful technique for handling memory limitations.  This is the recommended approach for splitting the dataset into mini-batches for training a neural network, rather than creating all the data splits upfront and keeping them in memory during training.

## Embedding

During the data preparation in the previous step, we generated sequences of the same length.  The elements of these sequences were integer numbers that corresponded to the _indices_ of unique words.

These words indices can be converted into input features in several different ways.  One naïve way is to apply one-hot encoding to convert indices into vectors of zeros and ones.  Then each word will be mapped to a vector whose size is the number of unique words in the entire dataset.  But a model trained on such features may suffer from the **curse of dimensionality**.  Furthermore, these features are very sparse, since all are zero except one.

A more elegant way is to map each word to a vector of fixed size with real-valued elements (not necessarily integers).  In contrast to the one-hot encoded vectors, we can use finite-sized vectors to represent an infinite amount of real numbers (in theory, we can extract infinite real numbers from a given interval, for example $[-1,1]$).

This is the idea behind embedding, which is a feature-learning technique to automatically learn the salient features to represent the words in our dataset.  Given the number of unique words _unique\_words_, we can choose the size of the embedding vectors to be much smaller than the number of unique words to represent the entire vocabulary.

The advantages of embedding over one-hot encoding are as follows:
- A reduction in the dimensionality of the feature space to decrease the effect of the curse of dimensionality.
- The extraction of salient features since the embedding layer in a neural network is trainable.

The following schematic representation shows how embedding works by mapping vocabulary indices to a trainable embedding matrix:

<img src="images/16_10.png" style="width:500px">

TensorFlow implements an efficient function, `tf.nn.embedding_lookup` that maps each integer that corresponds to a unique word, to a row of this trainable matrix.  For example, integer 1 is mapped to the first row, integer 2 is mapped to the second row, and so on.  Then, given a sequence of integers, such as $[0, 5, 3, 4, 19, 2, \ldots]$, we need to look up the corresponding rows for each element of this sequence.

Now, let us see how we can create an embedding layer in practice.  If we have `tf_x` as the input layer where the corresponding vocabulary indices are fed with type `tf.int32`, then creating an embedding layer can be done in two ste3ps, as follows:
1. We start by creating a matrix of size $[\mathrm{n\_words}\times\mathrm{embedding\_size}]$ as a tensor variable, which we call embedding, and we initialise its elements randomly with floats between $[-1, 1]$:
```python
embedding = tf.Variable(tf.random_uniform(shape=(n_words, embedding_size), minval=-1, maxval=1))
```
2. Then, we use the `tf.nn.embedding_lookup` function to look up the row in the embedding matrix associated with each element of `tf_x`:
```python
embed_x = tf.nn.embedding_lookup(embedding, tf_x)
```

Notice that `tf.nn.embedding_lookup` requires 2 mandatory arguments: the embedding tensor, and the lookup IDs.

## Building an RNN model

Now we're ready to build an RNN model.  We'll implement a `SentimentRNN` class that has the following methods:
- A constructor to set all the model parameters, and then create a computation graph, and call the `self.build` method to build the multilayer RNN model.
- A `build` method that declares three placeholders for input data, input labels, and the keep probability for the dropout configuration of the hidden layer.  After declaring these, it creates an embedding layer, and builds the multilayer RNN using the embedded representation as the input.
- A `train` method that creates a TensorFlow session for launching the computation graph, iterates through the mini-batches of data, and runs for a fixed number of epochs, to minimise the cost function defined in the graph.  This method also saves the model after 10 epochs for checkpointing.
- A `predict` method that creates a new session, restores the last checkpoint saved during the training process, and carries out the predictions for the test data.

In the following code, we'll see the implementation of this class and its methods broken into separate code sections.

## The SentimentRNN class Constructor

Let us start with the constructor of our `SentimentRNN` class, which we'll code as follows:

In [32]:
import tensorflow as tf

class SentimentRNN(object):
    def __init__(self, n_words, seq_len=200, lstm_size=256, num_layers=1,
                batch_size=64, learning_rate=0.0001, embed_size=200):
        self.n_words = n_words
        self.seq_len = seq_len
        self.lstm_size = lstm_size  # the number of hidden units
        self.num_layers = num_layers
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.embed_size = embed_size
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            self.build()
            self.saver = tf.train.Saver()
            self.init_op = tf.global_variables_initializer()
    
    """Please refer to the Markdown cell below: 'The build method'"""
    def build(self):
        # Define the placeholders
        tf_x = tf.placeholder(tf.int32, shape=(self.batch_size, self.seq_len), name="tf_x")
        tf_y = tf.placeholder(tf.float32, shape=(self.batch_size), name="tf_y")
        tf_keepprob = tf.placeholder(tf.float32, name="tf_keepprob")
        
        # Creating the embedding layer
        embedding = tf.Variable(tf.random_uniform((self.n_words, self.embed_size), minval=-1, maxval=1),
                               name="embedding")
        embed_x = tf.nn.embedding_lookup(embedding, tf_x, name='embedded_x')
        
        # Define the LSTM cell and stack them together  tf.nn.rnn_cell.LSTMCell
        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(
        self.lstm_size), output_keep_prob=tf_keepprob) for i in range(self.num_layers)])
        
        # Define the initial state:
        self.initial_state = cells.zero_state(self.batch_size, tf.float32)
        print('  << initial state >>  ', self.initial_state)
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, embed_x, initial_state=self.initial_state)
        
        # Note: lstm_outputs shape = [batch_size, max_time, cells.output_size]
        print("\n << lstm_output >> ", lstm_outputs)
        print("\n << final state >> ", self.final_state)
        
        logits = tf.layers.dense(inputs=lstm_outputs[:, -1], units=1, activation=None, name="logits")
        
        logits = tf.squeeze(logits, name="logits_squeezed")
        print("\n << logits >> ", logits)
        
        y_proba = tf.nn.sigmoid(logits, name="probabilities")
        predictions = {"probabilities": y_proba,
                      "labels": tf.cast(tf.round(y_proba), tf.int32, name="labels")}
        print("\n << predictions >> ", predictions)
        
        # Define the cost function
        cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels=tf_y, logits=logits), name="cost")
        
        # Define the optimizer
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.minimize(cost, name="train_op")
    
    def train(self, X_train, y_train, num_epochs):
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            iteration = 1
            for epoch in range(num_epochs):
                state = sess.run(self.initial_state)
                
                for batch_x, batch_y in create_batch_generator(X_train, y_train, self.batch_size):
                    feed = {"tf_x:0": batch_x, "tf_y:0": batch_y, "tf_keepprob:0": 0.5, self.initial_state: state}
                    loss, _, state = sess.run(["cost:0", "train_op", self.final_state], feed_dict=feed)
                    
                    if not iteration % 20:
                        print("Epoch: {}/{} Iteration: {} | Train loss: {:.5f}".format(epoch + 1,
                                                                                       num_epochs,
                                                                                       iteration,
                                                                                       loss))
                    iteration += 1
                if not (epoch + 1) % 10:
                    self.saver.save(sess, "model/sentiment-{}.ckpt".format(epoch))
    
    def predict(self, X_data, return_proba=False):
        preds = []
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess, tf.train.latest_checkpoint("./model/"))
            test_state = sess.run(self.initial_state)
            for ii, batch_x in enumerate(create_batch_generator(X_data, None, batch_size=self.batch_size), 1):
                feed = {"tf_x:0": batch_x, "tf_keepprob:0": 1.0, self.initial_state: test_state}
                if return_proba:
                    pred, test_state = sess.run(["probabilities:0", self.final_state], feed_dict=feed)
                else:
                    pred, test_state = sess.run(["labels:0", self.final_state], feed_dict=feed)
                
                preds.append(pred)
        
        return np.concatenate(preds)

- `n_words` must be set equal to the number of unique words (plus 1 since we use zero to fill sequences whose size is less than 200).  It is used while creating the embedding layer along with the `embed_size` hyperparameter.
- `seq_len` variable must be set according to the length of the sequences that were created in the preprocessing steps we went through previously.
- `lstm_size` is another hyperparameter that determines the number of hidden units in each RNN layer.

## The build method

Next, let us discuss the `build` method for our `SentimentRNN` class.  This is the longest and most critical method in our sequence, so we'll be going through it in plenty of detail.  First we'll look at the code in full, so we can see the whole picture, and then we'll analyze each of its main parts.

So first of all in our `build` method here, we created three placeholders, namely `tf_x`, `tf_y`, and `tf_keepprob`, which we need for feeding the input data.  Then we aded the embedding layer, which builds the embedded representation `embed_x`, as we discussed earlier.

Next, in our `build` method, we built the RNN network with LSTM cells.  We did this in three steps:
1. First, we defined the multilayer RNN cells.
2. Next, we defined the initial state for these cells.
3. Finally, we created an RNN specified by the RNN cells and their initial states.

Let's break these three steps out in deail in the following three sections, so we can examine in depth how we built the RNN network in our `build` method.

### Step 1---Defining multilayer RNN cells.

To examine how we coded our `build` method to build the RNN network, the first step was to define our multilayer RNN cells.

Fortunately, TensorFlow has a very nice wrapper class to define LSTM cells---the `BasicLSTMCell` class---which can be stacked together to from a multilayer RNN using the `MultiRNNCell` wrapper class.  The process of stacking RNN cells with a dropout has three nested steps; these three nested steps can be described from the inside out as follows:
1. First, create the RNN cells using `tf.contrib.rnn.BasicLSTMCell`.
2. Apply the dropout to the RNN cells using `tf.contrib.rnn.DropoutWrapper`.
3. Make a list of such cells according to the desired number of RNN layers, and pass this list to `tf.contrib.rnn.MultiRNNCell`.

In our `build` method code, this list is created using Python list comprehension.  Note that for a single layer, this list has oly one cell.

### Step 2---Defining the initial states for the RNN cells.

The second step that our `build` method takes to build the RNN network was to define the initial states for the RNN cells.

You will recall from the architecture of LSTM cells, there are 3 types of inputs in an LSTM cell---input data $\mathbf{x}^{(t)}$, activations of hidden units from the previous time step $\mathbf{h}^{(t-1)}$, and the cell state from the previous time step $\mathbf{C}^{(t-1)}$.

Therefore, in our `build` method implementation, $\mathbf{x}^{(t)}$ is the embedded `embed_x` data tensor.  When we start processing a new input sequence, we initialize the cell states to zero state; then after each time step we need to store the updated state of the cells to use for the next time step.

Once our multilayer RNN object is defined (`cells` in our implementation), we define its initial state in our `build` method using the `cells.zero_state` method.

### Step 3---Creating the RNN using the RNN cells and their states.

The third step to creating the RNN in our `build` method, used the `tf.nn.dynamic_rnn` function to pull together all our components.

The `tf.nn.dynamic_rnn` function therefore pulls the embedded data, the RNN cells, and their initial states, and creates a pipeline for them according to the unrolled architecture of LSTM cells.

The `tf.nn.dynamic_rnn` functionreturns a tuple containing the activations of the RNN cells, `outputs`; and their final states, `state`.  The output is a three-dimensional tensor with the shape `(batch_size, num_steps, lstm_size)`.  We pass `outputs` to a fully connected layer to get logits, and we store the final state to use as the initial state of the next mini-batch of data.

Finally, in our build method, after setting up the RNN components of the network, the cost function and optimization schemes can be defined like any other neural network.

## The train method

The next method in our `SentimentRNN` class is `train`.  This method is quite similar to the train methods we created in _Chapter 14_ and _Chapter 15_, except that we have an additional tensor, `state`, that we feed into our network.

The code above shows the implementation of the `train` method...

In this implementation of our `train` method, at the beginning of each epoch, we start from the zero states of RNN cells as our current state.  Running each mini-batch of data is performed by feeding the current state along with the data `batch_x` and their labels `batch_y`.  Upon finishing the execution of a mini-batch, we update the state to be the final state, which is returned by the `tf.nn.dynamic_rnn` function.  This updated state will be used toward execution of the next mini-batch.  This process is repeated and the current state is updated throughout the epoch.

## The predict method

Finally, the last method in our `SentimentRNN` class is the `predict` method, which keeps updating the current state similar to the `train` method, shown in the code above...

## Instantiating the SentimentRNN class

We've now coded and examined all four parts of our `SentimentRNN` class, which were the class constructor, the `build` method, the `train` method, and the `predict` method.

We are now ready to create an object of the class `SentimentRNN`, with parameters as follows:

In [33]:
n_words = max(list(word_to_int.values())) + 1

rnn = SentimentRNN(n_words=n_words,
                  seq_len=sequence_length,
                  embed_size=256,
                  lstm_size=128,
                  num_layers=1,
                  batch_size=100,
                  learning_rate=0.001)

  << initial state >>   (LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/LSTMCellZeroState/zeros:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/DropoutWrapperZeroState/LSTMCellZeroState/zeros_1:0' shape=(100, 128) dtype=float32>),)

 << lstm_output >>  Tensor("rnn/transpose_1:0", shape=(100, 200, 128), dtype=float32)

 << final state >>  (LSTMStateTuple(c=<tf.Tensor 'rnn/while/Exit_3:0' shape=(100, 128) dtype=float32>, h=<tf.Tensor 'rnn/while/Exit_4:0' shape=(100, 128) dtype=float32>),)

 << logits >>  Tensor("logits_squeezed:0", shape=(100,), dtype=float32)

 << predictions >>  {'probabilities': <tf.Tensor 'probabilities:0' shape=(100,) dtype=float32>, 'labels': <tf.Tensor 'labels:0' shape=(100,) dtype=int32>}


Notice here that we use `num_layers=1` to use a single RNN layer.  Although our implementation allows us to create multilayer RNNs, by setting `num_layers` greater than 1.  Here we should consider the small size of our dataset, and that a single RNN may generalise better to unseen data, since it is less likely to overfit the training data.

## Training and optimising the sentiment analysis RNN model

Next, we can train the RNN model by calling the `rnn.train` function.  In the following code, we train the model for `40` epochs, using the input from `X_train` and the corresponding class labels stored in `y_train`:

In [34]:
rnn.train(X_train, y_train, num_epochs=40)

Epoch: 1/40 Iteration: 20 | Train loss: 0.70071
Epoch: 1/40 Iteration: 40 | Train loss: 0.63536
Epoch: 1/40 Iteration: 60 | Train loss: 0.67273
Epoch: 1/40 Iteration: 80 | Train loss: 0.57852
Epoch: 1/40 Iteration: 100 | Train loss: 0.55505
Epoch: 1/40 Iteration: 120 | Train loss: 0.53757
Epoch: 1/40 Iteration: 140 | Train loss: 0.59300
Epoch: 1/40 Iteration: 160 | Train loss: 0.48948
Epoch: 1/40 Iteration: 180 | Train loss: 0.52614
Epoch: 1/40 Iteration: 200 | Train loss: 0.55652
Epoch: 1/40 Iteration: 220 | Train loss: 0.40346
Epoch: 1/40 Iteration: 240 | Train loss: 0.46770
Epoch: 2/40 Iteration: 260 | Train loss: 0.49523
Epoch: 2/40 Iteration: 280 | Train loss: 0.43171
Epoch: 2/40 Iteration: 300 | Train loss: 0.40109
Epoch: 2/40 Iteration: 320 | Train loss: 0.43319
Epoch: 2/40 Iteration: 340 | Train loss: 0.45512
Epoch: 2/40 Iteration: 360 | Train loss: 0.30557
Epoch: 2/40 Iteration: 380 | Train loss: 0.41766
Epoch: 2/40 Iteration: 400 | Train loss: 0.36661
Epoch: 2/40 Iteration: 4

Epoch: 14/40 Iteration: 3300 | Train loss: 0.00135
Epoch: 14/40 Iteration: 3320 | Train loss: 0.00058
Epoch: 14/40 Iteration: 3340 | Train loss: 0.04257
Epoch: 14/40 Iteration: 3360 | Train loss: 0.00193
Epoch: 14/40 Iteration: 3380 | Train loss: 0.05744
Epoch: 14/40 Iteration: 3400 | Train loss: 0.00159
Epoch: 14/40 Iteration: 3420 | Train loss: 0.06358
Epoch: 14/40 Iteration: 3440 | Train loss: 0.03425
Epoch: 14/40 Iteration: 3460 | Train loss: 0.05493
Epoch: 14/40 Iteration: 3480 | Train loss: 0.01376
Epoch: 14/40 Iteration: 3500 | Train loss: 0.00796
Epoch: 15/40 Iteration: 3520 | Train loss: 0.02557
Epoch: 15/40 Iteration: 3540 | Train loss: 0.00461
Epoch: 15/40 Iteration: 3560 | Train loss: 0.10629
Epoch: 15/40 Iteration: 3580 | Train loss: 0.04537
Epoch: 15/40 Iteration: 3600 | Train loss: 0.02648
Epoch: 15/40 Iteration: 3620 | Train loss: 0.04706
Epoch: 15/40 Iteration: 3640 | Train loss: 0.00507
Epoch: 15/40 Iteration: 3660 | Train loss: 0.01458
Epoch: 15/40 Iteration: 3680 | 

Epoch: 27/40 Iteration: 6520 | Train loss: 0.00005
Epoch: 27/40 Iteration: 6540 | Train loss: 0.00002
Epoch: 27/40 Iteration: 6560 | Train loss: 0.00008
Epoch: 27/40 Iteration: 6580 | Train loss: 0.00005
Epoch: 27/40 Iteration: 6600 | Train loss: 0.00001
Epoch: 27/40 Iteration: 6620 | Train loss: 0.00046
Epoch: 27/40 Iteration: 6640 | Train loss: 0.00001
Epoch: 27/40 Iteration: 6660 | Train loss: 0.00008
Epoch: 27/40 Iteration: 6680 | Train loss: 0.00003
Epoch: 27/40 Iteration: 6700 | Train loss: 0.00002
Epoch: 27/40 Iteration: 6720 | Train loss: 0.00001
Epoch: 27/40 Iteration: 6740 | Train loss: 0.00003
Epoch: 28/40 Iteration: 6760 | Train loss: 0.00002
Epoch: 28/40 Iteration: 6780 | Train loss: 0.00009
Epoch: 28/40 Iteration: 6800 | Train loss: 0.00001
Epoch: 28/40 Iteration: 6820 | Train loss: 0.00001
Epoch: 28/40 Iteration: 6840 | Train loss: 0.00010
Epoch: 28/40 Iteration: 6860 | Train loss: 0.00003
Epoch: 28/40 Iteration: 6880 | Train loss: 0.00009
Epoch: 28/40 Iteration: 6900 | 

Epoch: 39/40 Iteration: 9740 | Train loss: 0.00182
Epoch: 40/40 Iteration: 9760 | Train loss: 0.00098
Epoch: 40/40 Iteration: 9780 | Train loss: 0.00105
Epoch: 40/40 Iteration: 9800 | Train loss: 0.00419
Epoch: 40/40 Iteration: 9820 | Train loss: 0.00281
Epoch: 40/40 Iteration: 9840 | Train loss: 0.00177
Epoch: 40/40 Iteration: 9860 | Train loss: 0.00334
Epoch: 40/40 Iteration: 9880 | Train loss: 0.01384
Epoch: 40/40 Iteration: 9900 | Train loss: 0.00191
Epoch: 40/40 Iteration: 9920 | Train loss: 0.00464
Epoch: 40/40 Iteration: 9940 | Train loss: 0.00070
Epoch: 40/40 Iteration: 9960 | Train loss: 0.00126
Epoch: 40/40 Iteration: 9980 | Train loss: 0.00028
Epoch: 40/40 Iteration: 10000 | Train loss: 0.00082


The trained model is saved using TensorFlow's checkpointing system, which we discussed in Chapter 14.  Now we can use the trained model for predicting the class labels on the test set, as follows:

In [35]:
preds = rnn.predict(X_test)
y_true = y_test[:len(preds)]
print("Test accuracy:  {:.3f}.".format(np.sum(preds == y_true) / len(y_true)))

INFO:tensorflow:Restoring parameters from ./model/sentiment-39.ckpt
Test accuracy:  0.853.


The result will show an accuracy of 85 percent.  Given the small size of this dataset, this is comparable to the test prediction accuracy obtained in Chapter 8.

We can optimise this further by changing the hyperparameters of the model, such as `lstm_size`, `seq_len`, and `embed_size`, to achieve better generalisation performance.  However, for hyperparameter tuning, it is recommended that we create a separate validation set and that we don't repeatedly use the test set for evaluation to avoid introducing bias through test data leakage, which we discuess in Chapter 6.

Also, if you're interested in the prediction probabilities on the test set rather than the class labels, then you can set `return_proba=True` as follows:

In [36]:
proba = rnn.predict(X_test, return_proba=True)
print(proba)

INFO:tensorflow:Restoring parameters from ./model/sentiment-39.ckpt
[5.57116550e-07 9.99999046e-01 1.85828488e-02 ... 2.16272201e-06
 1.14251845e-04 9.95571196e-01]


So this was our first RNN model for sentiment analysis.  We'll now go further and create an RNN for character-bycharacter language modeling in TensorFlow, as another popular application of sequence modeling.

## Project two---Implementing an RNN for chatacter-level language modeling in TensorFlow.

Language modeling is a fascinating application that enables machines to perform human-language-related tasks, such as generating English sentences.  One of the most interesting efforts in this area is the work done by Sutskever, Martens, and Hinton (_Generating Text with Recurrent Neural Networks, Ilya Sutskever, James Martens, and Geoffrey E. Hinton, Proceedings of the 28th International Conference on Machine Learning (ICML-11), 2011_ https://pdfs.semanticscholar.org/93c2/0e38c85b69fc2d2eb314b3c1217913f7db11.pdf).

In the model that we'll build now, the input is a text document, and our goal is to develop a model that can generate new text siilar to the input document.  Examples of such an input can be a book or a computer program in a specific programming language.

In the character-level language modeling, the input is broken down into a sequence of characters that are fed into our network one characer at a time.  The network will process each new character in conjunction with the memory of the previously seen characters to predict the next character.  The following figure shows an example of character-level language modeling.

<img src="images/16_11.png" style="width:500px">

We can break this implementation down into three separate steps---preparing the data, building the RNN model, and performing next-character prediction and sampling to generate new text.

If you recall from the previous sections of this chapter, we mentioned the exploding gradient problem.  In this application, we'll also get a chance to play with a gradient clipping technique to avoid this exploding gradient problem.

## Preparing the Data

In this section, we prepare the data fro character-level language modeling.

To get the input data, visit the Project Gutenberg website at https://www.gutenberg.org/, which provides thousands of free e-books.  For our example, we can get the book _The Tragedie of Hamlet_ by William Shakespeare in plain text format from http://www.gutenberg.org/cache/epub/2265/pg2265.txt.

Note that this link will directly take you to the download page.  If you are using macOS or a Linux operating system, you can download the file with the following command in the Terminal:  
`curl https://www.gutenberg.org/cache/epub/2265/pg2265.txt > pg2265.txt`

If this resource becomes unavailable in the future, a copy of this text is also included in this chapter's code directory in the book's code repository at https://github.com/rasbt/python-machine-learning-book-2nd-edition.

Once we have some data, we can read it into a Python session as plain text.  In the following code, the Python variable `chars` represents the set of unique characters observed in this text.  We then create a dictionary that maps each character to an integer, `char2int`, and a dictionary that performs reverse mapping, for instance, mapping integers to those unique characters---`int2char`.  Using the `char2int` dictionary, we convert the text into a NumPy array of integers.  The following figure shows an example of converting characters into integers and the reverse for the words `"Hello"` and `"world"`:

<img src="images/16_12.png" style="width:500px">

The code below reads the text from the downloaded link, removes the beginning portion of the text that contains some legal description of the Gutenberg project, and then constructs the dictionaries based on the text:

In [47]:
import numpy as np

# Reading and processing the text
with open("1257.txt", "r", encoding="utf-8") as f:
    text = f.read()
# text = text[15858:]
chars = set(text)

char2int = {ch: i for i, ch in enumerate(chars)}
int2char = dict(enumerate(chars))
text_ints = np.array([char2int[ch] for ch in text], dtype=np.int32)

Now, we should reshape the data into batches of sequences, the most important step in preparing the data.  As we know, the goal is to predict the next character, based on the sequene of characters that we have observed so far.  Therefore, we shift the input $(\mathbf{x})$ and output $(\mathbf{y})$ of the neural network by one character.  The following figure shows the preprocessing steps, starting from a text corpus to generating data arrays for $\mathbf{x}$ and $\mathbf{y}$:

<img src="images/16_13.png" style="width:500px">

As you can see in this figure, the training arrays $\mathbf{x}$ and $\mathbf{y}$ have the same shapes or dimensions, where the number of rows is equal to the _batch size_ and the number of columns is equal to _number of batches$\times$number of steps_.

Given the input array `data` that contains the integers that correspond to the characters in the text corpus, the following function will generate `x` and `y` with the same structure shown in the previous figure.

In [48]:
def reshape_data(sequence, batch_size, num_steps):
    tot_batch_length = batch_size * num_steps
    num_batches = int(len(sequence) / tot_batch_length)
    if num_batches*tot_batch_length + 1 > len(sequence):
        num_batches = num_batches - 1
    
    # Truncatethe sequence at the end to get rid of the remaining characters that do not make a full batch
    x = sequence[0: num_batches*tot_batch_length]
    y = sequence[1: num_batches*tot_batch_length + 1]
    
    # Split x and y into a list batches of sequences:
    x_batch_splits = np.split(x, batch_size)
    y_batch_splits = np.split(y, batch_size)
    
    # Stack the batches together batch_size x tot_batch_length
    x = np.stack(x_batch_splits)
    y = np.stack(y_batch_splits)
    
    return x, y

The next step is to split the arrays $\mathbf{x}$ and $\mathbf{y}$ into mini-batches, where each row is a sequence with length equal to the _number of steps_.  The process of splitting the data array $\mathbf{x}$ is shown in the following figure:

<img src="images/16_14.png" style="width:500px">

In the following code, we define a function named `create_batch_generator` that splits the data arrays $\mathbf{x}$ and $\mathbf{y}$, as shown in the previous figure, and outputs a batch generator.  Later, we will use this generator to iterate through the mini-batches during the training of our network:

In [49]:
def create_batch_generator(data_x, data_y, num_steps):
    batch_size, tot_batch_length = data_x.shape
    num_batches = int(tot_batch_length / num_steps)
    for b in range(num_batches):
        yield (data_x[:, b*num_steps:(b+1)*num_steps],
               data_y[:, b*num_steps:(b+1)*num_steps])

At this point, we've now completed the data preprocessing steps, and we have the data in the proper format.  In the next section, we'll implement the RNN model for character-level language modeling.

## Building a character-level RNN model

To build a character-level neural network, we'll implement a class called `CharRNN` that constructs the graph of the RNN in order to predict the next character, after observing a given sequence of characters.  From the classification perspective, the number of classes is the total number of unique characters that exists in the text corpus.  The `CharRNN` class has four method, as follows:
- A constructor that sets up the learning parameters, creates a computation graph, and calls the `build` method to construct the graph based on the sampling mode versus the training mode.
- A `build` method that defines the placeholders for feeding the data, constructs the RNN using LSTM cells, and defines the output of the network, the cost function, and the optimizer.
- A `train` method to iterate through the mini-batches, and train the network for the specified number of epochs.
- A `sample` method to start from a given string, calculate the probabilities for the next character, and choose a character randomly according to these probabilities.  This process will be repeated, and the sampled characters will be concatenated together to form a string.  Once the size of this string reaches the specified length, it will return the string.

We'll break these four methods into separate code sections and explain each one.  Note that implementing the RNN part of this model is very similar to the implementation in the _Project one---Performing sentiment analysis of IMDb movie reviews using multilayer RNNs_ section.  So, we'll skip the description of building the RNN components here.

## Some code that needs to be defined now; this is explained at the end.

In this `get_top_char` function, the probabilities are first sorted, then the `top_n` probabilities are passed to the `numpy.random.choice` function to randomly select one out of these top probabilities.  The implementation of the `get_top_char` function is as follows.

Note, of course, that this function should be defined _before_ the definition of the `CharRNN` class.

In [50]:
def get_top_char(probas, char_size, top_n=5):
    p = np.squeeze(probas)
    p[np.argsort(p)[:-top_n]] = 0.0
    p = p / np.sum(p)
    ch_id = np.random.choice(char_size, 1, p=p)[0]
    return ch_id

## The constructor

In contrast to our previous implementation for sentiment analysis, where the same computation graph was used for both training and prediction modes, this time our computation graph is going to be different for the training versus sampling mode.

Therefore, we need to add a new Boolean type argument to the constructor, to determine whether we're building the model for the training mode or the sampling mode.  The following code shows the implementation of the constructor enclosed in the class definition, amongst others...

As we planned earlier, the Boolean `sampling` argument is used to determine whether the instance of `CharRNN` is for building the graph in the training mode (`sampling=False`), or the sampling mode (`sampling=True`).

In addition to the `sampling` argument, we've introduced a new argument called `grad_clip`, which is used for clipping the gradients to avoid the exploding gradient problem that we mentioned earlier.

Then, similar to the previous implementation, the constructor creates a computation graph, sets the graph-level random seed, for consistent output, and build the graph by calling the `build` method.

## The build method

The next method of the `CharRNN` class is `build`, which is very similar to the `build` method in the _Project one---performing sentiment analysis of IMDb movie reviews using multilayer RNNs_ section, except for some minor differences.  The `build` method first defines two local variables, `batch_size`, and `num_steps`, based on the mode, as follows:

$$
\mathrm{in\ sampling\ mode}:
\begin{cases}
    \mathrm{batch\_size} = 1 \\
    \mathrm{num\_steps} = 1
\end{cases}
$$

$$
\mathrm{in\ training\ mode}:
\begin{cases}
    \mathrm{batch\_size} = \mathrm{self.batch\_size} \\
    \mathrm{num\_steps} = \mathrm{self.num\_steps}
\end{cases}
$$

Recall that in the sentiment analysis implementation, we used an embedding layer to create a salient representation for the unique words in the dataset.  In contrast, here we are using the one-hot encoding scheme for both $x$ and $y$ with `depth=num_classes`, where `num_classes` is in fact the total number of characters in the text corpus.

Building a multilayer RNN component of the model is exactly the same as in our sentiment analysis implementation, using the `tf.nn.dynamic_rnn` function.  However, `outputs` from the `tf.nn.dynamic_nn` function is a three-dimensional tensor with this shape---`batch_size, num_steps, lstm_size`.  Next, this tensor will be reshaped into a two-dimensional tensor with the `batch_size*num_steps, lstm_size` shape, which is passed to the `tf.layers.dense` function to make a fully connected layer and obtain `logits` (net inputs).  Finally, the probabilities for the next batch of characters are obtained and the cost function is defined.  In addition, here, we apply gradient clipping using the `tf.clip_by_global_norm` function to avoid the exploding gradient problem.

## The train method

The next method of the `CharRNN` class is the `train` method, which is very similar to the `train` method described in the `Project one---performing sentiment analysis of IMDb movie reviews using multilayer RNNs` section.

##  The sample method

The final method in our `CharRNN` class is the `sample` method.  The behavior of this `sample` method is similar to that of the `predict` method we implemented in the `Project one---performing sentiment analysis of IMDb movie review using multilayer RNNs` section.  However, the difference here is that we calculate the probabilities for the next character from an observed sequence---`observed_seq`.  Then, these probabilities are passes to a function named `get_top_char`, which randomly selects one character acording to the obtained probabilities.

Initially, the observed sequence starts from `starter_seq`, which is provided as an argument.  When new characters are sampled according to their predicted probabilities, they are appended to the observed sequence, and the new observed sequence is used for predicting the next character.

So here, the `sample` method calls the `get_top_char` function to choose a character ID randomly (`ch_id`) according to the obtained probabilities.

In this `get_top_char` function, the probabilities are first sorted, then the `top_n` probabilities are passed to the `numpy.random.choice` function to randomly select one out of these probabilities.  

In [51]:
import tensorflow as tf
import os

class CharRNN(object):
    def __init__(self, num_classes, batch_size=64, num_steps=100, lstm_size=128,
                num_layers=1, learning_rate=0.001, keep_prob=0.5, grad_clip=5, sampling=False):
        self.num_classes = num_classes
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.learning_rate = learning_rate
        self.keep_prob = keep_prob
        self.grad_clip = grad_clip
        
        self.g = tf.Graph()
        with self.g.as_default():
            tf.set_random_seed(123)
            
            self.build(sampling=sampling)
            
            self.saver = tf.train.Saver()
            
            self.init_op = tf.global_variables_initializer()
    
    def build(self, sampling):
        if sampling:
            batch_size, num_steps = 1, 1
        else:
            batch_size = self.batch_size
            num_steps = self.num_steps
        
        tf_x = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name="tf_x")
        tf_y = tf.placeholder(tf.int32, shape=[batch_size, num_steps], name="tf_y")
        tf_keepprob = tf.placeholder(tf.float32, name="tf_keepprob")
        
        # One-hot encoding
        x_onehot = tf.one_hot(tf_x, depth=self.num_classes)
        y_onehot = tf.one_hot(tf_y, depth=self.num_classes)
        
        # Build the multi-layer RNN cells
        cells = tf.contrib.rnn.MultiRNNCell([tf.contrib.rnn.DropoutWrapper(tf.nn.rnn_cell.LSTMCell(
        self.lstm_size), output_keep_prob=tf_keepprob) for _ in range(self.num_layers)])
        
        # Define the initial state
        self.initial_state = cells.zero_state(batch_size, tf.float32)
        
        # Run each sequence step through the RNN
        lstm_outputs, self.final_state = tf.nn.dynamic_rnn(cells, x_onehot, initial_state=self.initial_state)
        
        print("  << lstm_outputs >>", lstm_outputs)
        
        seq_output_reshaped = tf.reshape(lstm_outputs, shape=[-1, self.lstm_size], name="seq_out_reshaped")
        
        logits = tf.layers.dense(inputs=seq_output_reshaped, units=self.num_classes,
                                 activation=None, name="logits")
        
        proba = tf.nn.softmax(logits, name="probabilities")
        
        y_reshaped = tf.reshape(y_onehot, shape=[-1, self.num_classes], name="y_reshaped")
        
        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped),
                             name="cost")
        
        # Gradient clipping to avoid "exploding gradient"
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), self.grad_clip)
        optimizer = tf.train.AdamOptimizer(self.learning_rate)
        train_op = optimizer.apply_gradients(zip(grads, tvars), name="train_op")
    
    def train(self, train_x, train_y, num_epochs, ckpt_dir="./model/"):
        # Create the checkpoint directory if it does not exist
        if not os.path.exists(ckpt_dir):
            os.mkdir(ckpt_dir)
        
        with tf.Session(graph=self.g) as sess:
            sess.run(self.init_op)
            
            n_batches = int(train_x.shape[1]/self.num_steps)
            iterations = n_batches * num_epochs
            for epoch in range(num_epochs):
                
                # Train the network
                new_state = sess.run(self.initial_state)
                loss = 0
                # Mini-batch generator
                bgen = create_batch_generator(train_x, train_y, self.num_steps)
                for b, (batch_x, batch_y) in enumerate(bgen, 1):
                    iteration = epoch * n_batches + b
                    
                    feed = {"tf_x:0": batch_x, "tf_y:0": batch_y, "tf_keepprob:0": self.keep_prob,
                           self.initial_state: new_state}
                    batch_cost, _, new_state = sess.run(["cost:0", "train_op", self.final_state], feed_dict=feed)
                    if not iteration % 10:
                        print("Epoch {}/{} Iteration {} | Training Loss: {:.4f}.".format(epoch + 1, num_epochs,
                                                                                        iteration, batch_cost))
                
                # Save the trained model
                self.saver.save(sess, os.path.join(ckpt_dir, "language_modeling.ckpt"))
    
    def sample(self, output_length, ckpt_dir, starter_seq="Men "):
        observed_seq = [ch for ch in starter_seq]
        with tf.Session(graph=self.g) as sess:
            self.saver.restore(sess, tf.train.latest_checkpoint(ckpt_dir))
            
            # 1:  Run the model using the starter seq
            new_state = sess.run(self.initial_state)
            for ch in starter_seq:
                x = np.zeros((1, 1))
                x[0, 0] = char2int[ch]
                feed = {"tf_x:0": x, "tf_keepprob:0": 1.0, self.initial_state: new_state}
                proba, new_state = sess.run(["probabilities:0", self.final_state], feed_dict=feed)
            
            ch_id = get_top_char(proba, len(chars))
            observed_seq.append(int2char[ch_id])
            
            # 2:  Run the model using the updated ovserved_seq
            for i in range(output_length):
                x[0, 0] = ch_id
                feed = {"tf_x:0": x, "tf_keepprob:0": 1.0, self.initial_state: new_state}
                proba, new_state = sess.run(["probabilities:0", self.final_state], feed_dict=feed)
                
                ch_id = get_top_char(proba, len(chars))
                observed_seq.append(int2char[ch_id])
        return "".join(observed_seq)

## Creating and Training the CharRNN Model.

Now we are ready to create an instance of the `CharRNN` class to build the RNN model, and to train it with the following configurations:

In [52]:
batch_size = 64
num_steps = 100
train_x, train_y = reshape_data(text_ints, batch_size, num_steps)

rnn = CharRNN(num_classes=len(chars), batch_size=batch_size)
rnn.train(train_x, train_y, num_epochs=500, ckpt_dir="./model-500/")

  << lstm_outputs >> Tensor("rnn/transpose_1:0", shape=(64, 100, 128), dtype=float32)
Epoch 1/500 Iteration 10 | Training Loss: 3.9287.
Epoch 1/500 Iteration 20 | Training Loss: 3.4434.
Epoch 1/500 Iteration 30 | Training Loss: 3.3455.
Epoch 1/500 Iteration 40 | Training Loss: 3.2452.
Epoch 1/500 Iteration 50 | Training Loss: 3.2319.
Epoch 1/500 Iteration 60 | Training Loss: 3.2077.
Epoch 1/500 Iteration 70 | Training Loss: 3.1810.
Epoch 1/500 Iteration 80 | Training Loss: 3.1811.
Epoch 1/500 Iteration 90 | Training Loss: 3.1635.
Epoch 1/500 Iteration 100 | Training Loss: 3.1477.
Epoch 1/500 Iteration 110 | Training Loss: 3.1109.
Epoch 1/500 Iteration 120 | Training Loss: 3.1414.
Epoch 1/500 Iteration 130 | Training Loss: 3.0823.
Epoch 1/500 Iteration 140 | Training Loss: 3.0749.
Epoch 1/500 Iteration 150 | Training Loss: 3.0285.
Epoch 1/500 Iteration 160 | Training Loss: 3.0779.
Epoch 1/500 Iteration 170 | Training Loss: 2.9796.
Epoch 1/500 Iteration 180 | Training Loss: 2.9406.
Epoch

Epoch 8/500 Iteration 1590 | Training Loss: 2.0520.
Epoch 8/500 Iteration 1600 | Training Loss: 2.0289.
Epoch 8/500 Iteration 1610 | Training Loss: 2.0188.
Epoch 8/500 Iteration 1620 | Training Loss: 2.0283.
Epoch 8/500 Iteration 1630 | Training Loss: 2.0323.
Epoch 9/500 Iteration 1640 | Training Loss: 1.9939.
Epoch 9/500 Iteration 1650 | Training Loss: 1.9372.
Epoch 9/500 Iteration 1660 | Training Loss: 1.9978.
Epoch 9/500 Iteration 1670 | Training Loss: 1.9892.
Epoch 9/500 Iteration 1680 | Training Loss: 2.0404.
Epoch 9/500 Iteration 1690 | Training Loss: 1.9815.
Epoch 9/500 Iteration 1700 | Training Loss: 2.0077.
Epoch 9/500 Iteration 1710 | Training Loss: 2.0754.
Epoch 9/500 Iteration 1720 | Training Loss: 1.9865.
Epoch 9/500 Iteration 1730 | Training Loss: 1.9849.
Epoch 9/500 Iteration 1740 | Training Loss: 1.9847.
Epoch 9/500 Iteration 1750 | Training Loss: 1.9424.
Epoch 9/500 Iteration 1760 | Training Loss: 1.9873.
Epoch 9/500 Iteration 1770 | Training Loss: 1.9932.
Epoch 9/500 

Epoch 16/500 Iteration 3150 | Training Loss: 1.7852.
Epoch 16/500 Iteration 3160 | Training Loss: 1.7855.
Epoch 16/500 Iteration 3170 | Training Loss: 1.7465.
Epoch 16/500 Iteration 3180 | Training Loss: 1.7764.
Epoch 16/500 Iteration 3190 | Training Loss: 1.7605.
Epoch 16/500 Iteration 3200 | Training Loss: 1.7830.
Epoch 16/500 Iteration 3210 | Training Loss: 1.7968.
Epoch 16/500 Iteration 3220 | Training Loss: 1.8351.
Epoch 16/500 Iteration 3230 | Training Loss: 1.7830.
Epoch 16/500 Iteration 3240 | Training Loss: 1.7855.
Epoch 16/500 Iteration 3250 | Training Loss: 1.8077.
Epoch 16/500 Iteration 3260 | Training Loss: 1.8112.
Epoch 17/500 Iteration 3270 | Training Loss: 1.8045.
Epoch 17/500 Iteration 3280 | Training Loss: 1.7947.
Epoch 17/500 Iteration 3290 | Training Loss: 1.7890.
Epoch 17/500 Iteration 3300 | Training Loss: 1.7635.
Epoch 17/500 Iteration 3310 | Training Loss: 1.8201.
Epoch 17/500 Iteration 3320 | Training Loss: 1.7686.
Epoch 17/500 Iteration 3330 | Training Loss: 1

Epoch 24/500 Iteration 4700 | Training Loss: 1.6788.
Epoch 24/500 Iteration 4710 | Training Loss: 1.6191.
Epoch 24/500 Iteration 4720 | Training Loss: 1.6954.
Epoch 24/500 Iteration 4730 | Training Loss: 1.6932.
Epoch 24/500 Iteration 4740 | Training Loss: 1.7583.
Epoch 24/500 Iteration 4750 | Training Loss: 1.6929.
Epoch 24/500 Iteration 4760 | Training Loss: 1.7391.
Epoch 24/500 Iteration 4770 | Training Loss: 1.7522.
Epoch 24/500 Iteration 4780 | Training Loss: 1.6975.
Epoch 24/500 Iteration 4790 | Training Loss: 1.6887.
Epoch 24/500 Iteration 4800 | Training Loss: 1.6882.
Epoch 24/500 Iteration 4810 | Training Loss: 1.6324.
Epoch 24/500 Iteration 4820 | Training Loss: 1.7009.
Epoch 24/500 Iteration 4830 | Training Loss: 1.7017.
Epoch 24/500 Iteration 4840 | Training Loss: 1.6450.
Epoch 24/500 Iteration 4850 | Training Loss: 1.6985.
Epoch 24/500 Iteration 4860 | Training Loss: 1.7043.
Epoch 24/500 Iteration 4870 | Training Loss: 1.6890.
Epoch 24/500 Iteration 4880 | Training Loss: 1

Epoch 31/500 Iteration 6250 | Training Loss: 1.5915.
Epoch 31/500 Iteration 6260 | Training Loss: 1.6144.
Epoch 31/500 Iteration 6270 | Training Loss: 1.6392.
Epoch 31/500 Iteration 6280 | Training Loss: 1.6704.
Epoch 31/500 Iteration 6290 | Training Loss: 1.6262.
Epoch 31/500 Iteration 6300 | Training Loss: 1.6363.
Epoch 31/500 Iteration 6310 | Training Loss: 1.6296.
Epoch 31/500 Iteration 6320 | Training Loss: 1.6295.
Epoch 32/500 Iteration 6330 | Training Loss: 1.6538.
Epoch 32/500 Iteration 6340 | Training Loss: 1.6223.
Epoch 32/500 Iteration 6350 | Training Loss: 1.6233.
Epoch 32/500 Iteration 6360 | Training Loss: 1.5982.
Epoch 32/500 Iteration 6370 | Training Loss: 1.6420.
Epoch 32/500 Iteration 6380 | Training Loss: 1.5985.
Epoch 32/500 Iteration 6390 | Training Loss: 1.6369.
Epoch 32/500 Iteration 6400 | Training Loss: 1.6757.
Epoch 32/500 Iteration 6410 | Training Loss: 1.6202.
Epoch 32/500 Iteration 6420 | Training Loss: 1.6067.
Epoch 32/500 Iteration 6430 | Training Loss: 1

...

Epoch 471/500 Iteration 96000 | Training Loss: 1.3468.
Epoch 471/500 Iteration 96010 | Training Loss: 1.3242.
Epoch 471/500 Iteration 96020 | Training Loss: 1.3427.
Epoch 471/500 Iteration 96030 | Training Loss: 1.3422.
Epoch 471/500 Iteration 96040 | Training Loss: 1.3827.
Epoch 471/500 Iteration 96050 | Training Loss: 1.3441.
Epoch 471/500 Iteration 96060 | Training Loss: 1.3802.
Epoch 471/500 Iteration 96070 | Training Loss: 1.3558.
Epoch 471/500 Iteration 96080 | Training Loss: 1.3690.
Epoch 472/500 Iteration 96090 | Training Loss: 1.3526.
Epoch 472/500 Iteration 96100 | Training Loss: 1.3552.
Epoch 472/500 Iteration 96110 | Training Loss: 1.3265.
Epoch 472/500 Iteration 96120 | Training Loss: 1.3393.
Epoch 472/500 Iteration 96130 | Training Loss: 1.3411.
Epoch 472/500 Iteration 96140 | Training Loss: 1.3181.
Epoch 472/500 Iteration 96150 | Training Loss: 1.3381.
Epoch 472/500 Iteration 96160 | Training Loss: 1.3626.
Epoch 472/500 Iteration 96170 | Training Loss: 1.3369.
Epoch 472/

Epoch 478/500 Iteration 97490 | Training Loss: 1.3516.
Epoch 478/500 Iteration 97500 | Training Loss: 1.3725.
Epoch 478/500 Iteration 97510 | Training Loss: 1.3916.
Epoch 479/500 Iteration 97520 | Training Loss: 1.3541.
Epoch 479/500 Iteration 97530 | Training Loss: 1.2838.
Epoch 479/500 Iteration 97540 | Training Loss: 1.3454.
Epoch 479/500 Iteration 97550 | Training Loss: 1.3400.
Epoch 479/500 Iteration 97560 | Training Loss: 1.4238.
Epoch 479/500 Iteration 97570 | Training Loss: 1.3518.
Epoch 479/500 Iteration 97580 | Training Loss: 1.3530.
Epoch 479/500 Iteration 97590 | Training Loss: 1.3708.
Epoch 479/500 Iteration 97600 | Training Loss: 1.3588.
Epoch 479/500 Iteration 97610 | Training Loss: 1.3383.
Epoch 479/500 Iteration 97620 | Training Loss: 1.3448.
Epoch 479/500 Iteration 97630 | Training Loss: 1.2971.
Epoch 479/500 Iteration 97640 | Training Loss: 1.3435.
Epoch 479/500 Iteration 97650 | Training Loss: 1.3567.
Epoch 479/500 Iteration 97660 | Training Loss: 1.2831.
Epoch 479/

Epoch 486/500 Iteration 98980 | Training Loss: 1.3765.
Epoch 486/500 Iteration 98990 | Training Loss: 1.3330.
Epoch 486/500 Iteration 99000 | Training Loss: 1.3630.
Epoch 486/500 Iteration 99010 | Training Loss: 1.3485.
Epoch 486/500 Iteration 99020 | Training Loss: 1.3169.
Epoch 486/500 Iteration 99030 | Training Loss: 1.3390.
Epoch 486/500 Iteration 99040 | Training Loss: 1.3534.
Epoch 486/500 Iteration 99050 | Training Loss: 1.2864.
Epoch 486/500 Iteration 99060 | Training Loss: 1.3380.
Epoch 486/500 Iteration 99070 | Training Loss: 1.3326.
Epoch 486/500 Iteration 99080 | Training Loss: 1.3504.
Epoch 486/500 Iteration 99090 | Training Loss: 1.3384.
Epoch 486/500 Iteration 99100 | Training Loss: 1.3846.
Epoch 486/500 Iteration 99110 | Training Loss: 1.3424.
Epoch 486/500 Iteration 99120 | Training Loss: 1.3818.
Epoch 486/500 Iteration 99130 | Training Loss: 1.3490.
Epoch 486/500 Iteration 99140 | Training Loss: 1.3735.
Epoch 487/500 Iteration 99150 | Training Loss: 1.3544.
Epoch 487/

Epoch 493/500 Iteration 100470 | Training Loss: 1.2937.
Epoch 493/500 Iteration 100480 | Training Loss: 1.3088.
Epoch 493/500 Iteration 100490 | Training Loss: 1.3363.
Epoch 493/500 Iteration 100500 | Training Loss: 1.2971.
Epoch 493/500 Iteration 100510 | Training Loss: 1.3201.
Epoch 493/500 Iteration 100520 | Training Loss: 1.3218.
Epoch 493/500 Iteration 100530 | Training Loss: 1.3801.
Epoch 493/500 Iteration 100540 | Training Loss: 1.3290.
Epoch 493/500 Iteration 100550 | Training Loss: 1.3372.
Epoch 493/500 Iteration 100560 | Training Loss: 1.3680.
Epoch 493/500 Iteration 100570 | Training Loss: 1.3856.
Epoch 494/500 Iteration 100580 | Training Loss: 1.3524.
Epoch 494/500 Iteration 100590 | Training Loss: 1.2745.
Epoch 494/500 Iteration 100600 | Training Loss: 1.3358.
Epoch 494/500 Iteration 100610 | Training Loss: 1.3315.
Epoch 494/500 Iteration 100620 | Training Loss: 1.4162.
Epoch 494/500 Iteration 100630 | Training Loss: 1.3404.
Epoch 494/500 Iteration 100640 | Training Loss: 

Epoch 500/500 Iteration 101940 | Training Loss: 1.3078.
Epoch 500/500 Iteration 101950 | Training Loss: 1.3459.
Epoch 500/500 Iteration 101960 | Training Loss: 1.3386.
Epoch 500/500 Iteration 101970 | Training Loss: 1.3773.
Epoch 500/500 Iteration 101980 | Training Loss: 1.3429.
Epoch 500/500 Iteration 101990 | Training Loss: 1.3138.
Epoch 500/500 Iteration 102000 | Training Loss: 1.3624.


## The CharRNN model in the sampling mode

Next up, we can create a new instance of the `CharRNN` class in the sampling mode by specifying that `sampling=True`.  We'll call the `sample` method to load the saved model in the `./model-250/` folder, and generate a sequence of 500 characters:

In [53]:
del rnn

np.random.seed(123)
rnn = CharRNN(len(chars), sampling=True)
print(rnn.sample(ckpt_dir="./model-500/", output_length=500))

  << lstm_outputs >> Tensor("rnn/transpose_1:0", shape=(1, 1, 128), dtype=float32)
INFO:tensorflow:Restoring parameters from ./model-500/language_modeling.ckpt
Men will ball to the person.
The mind and saying. To an
arrivel was his constantly he was till on the place at the cardinal, the stell tows the taken with his passess in the silenty and the thoughts stained on the service of suscetingen her secrece as he had shone is the partity with in that of the
sare; the charmen would not the took were bucking that she and the procurator’s wife of the crime the post, steps that he had someone the cast at himself to having, and the memenon to the tapponessenss, we


The generated text will look like the following:

<img src="images/16_15.png" style="width:500px">

You can see that in the resulting output, that some English words are mostly preserved.  It's also important to note that this is from an old English text, therefore, some words in the original text may be unfamiliar.  To get a better result, we would need to train for higher number of epochs.  Feel free to repeat this with a much larger document and train the model for more epochs.